In [1]:
import requests
import pandas as pd

In [2]:
def convert_c_to_f(celsius):
    if celsius is not None:
        return round((celsius * 9/5) + 32)
    return None

def convert_ms_to_mph(ms):
    if ms is not None:
        return round(ms * 2.23694)
    return None

def convert_pa_to_inhg(pa):
    if pa is not None:
        return round(pa * 0.0002953)
    return None

def convert_mm_to_inches(mm):
    if mm is not None:
        return round(mm * 0.0393701)
    return None

def convert_m_to_miles(m):
    if m is not None:
        return round(m * 0.000621371)
    return None

def convert_kmh_to_mph(kmh):
    if kmh is not None:
        return round(kmh * 0.621371)
    return None

In [3]:
# NWS LA/Oxnard office
url = 'https://api.weather.gov/offices/LOX'
# Perform the GET request
response = requests.get(url)

In [4]:
station_list = []

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()

    # Extract and print list of stations
    stations = data['approvedObservationStations']
    for station in stations:
        station_list.append(station.split('/')[4])
else:
    print(f"Failed to fetch data: {response.status_code}")

In [5]:
# API endpoint template for fetching latest observations for a station
observation_url_template = 'https://api.weather.gov/stations/{}/observations/latest'

data = []

for station in station_list:
    observation_url = observation_url_template.format(station)
    
    response = requests.get(observation_url)
    
    if response.status_code == 200:
        observation_data = response.json()['properties']
        
        # Extract and convert data
        temp_f = convert_c_to_f(observation_data['temperature']['value'])
        dewpoint_f = convert_c_to_f(observation_data['dewpoint']['value'])
        wind_speed_mph = convert_kmh_to_mph(observation_data['windSpeed']['value'])
        wind_gust_mph = convert_kmh_to_mph(observation_data['windGust']['value'])
        pressure_inhg = convert_pa_to_inhg(observation_data['barometricPressure']['value'])
        sea_level_pressure_inhg = convert_pa_to_inhg(observation_data['seaLevelPressure']['value'])
        visibility_miles = convert_m_to_miles(observation_data['visibility']['value'])
        precip_last_3h_inches = convert_mm_to_inches(observation_data['precipitationLast3Hours']['value'])
        max_temp_24h_f = convert_c_to_f(observation_data['maxTemperatureLast24Hours']['value'])
        min_temp_24h_f = convert_c_to_f(observation_data['minTemperatureLast24Hours']['value'])
        wind_chill_f = convert_c_to_f(observation_data['windChill']['value'])
        heat_index_f = convert_c_to_f(observation_data['heatIndex']['value'])

        # Append to data list
        data.append({
            'station_id': station,
            'timestamp': observation_data['timestamp'],
            'temp_f': temp_f,
            'dewpoint_f': dewpoint_f,
            'wind_speed_mph': wind_speed_mph,
            'precip_three_hours_inches': precip_last_3h_inches,
            'humidity_pct': observation_data['relativeHumidity']['value'],
            'heat_index_f': heat_index_f
        })
    else:
        print(f"Failed to fetch data for station {station}: {response.status_code}")

# Create a DataFrame
df = pd.DataFrame(data).dropna(subset=['temp_f', 'humidity_pct']).reset_index(drop=True)

Failed to fetch data for station WBPC1: 404


In [6]:
df

,station_id,timestamp,temp_f,dewpoint_f,wind_speed_mph,precip_three_hours_inches,humidity_pct,heat_index_f
0,AOMC1,2024-08-03T20:00:00+00:00,79.0,50.0,4.0,None,36.809063,78.0
1,BPNC1,2024-08-03T19:54:00+00:00,82.0,36.0,2.0,None,18.977200,80.0
2,CAZC1,2024-08-03T20:00:00+00:00,93.0,51.0,9.0,None,23.939153,90.0
3,CMOC1,2024-08-03T19:55:00+00:00,95.0,52.0,6.0,None,23.923627,92.0
4,CNIC1,2024-08-03T19:55:00+00:00,90.0,48.0,12.0,None,23.955429,87.0
5,CUUC1,2024-08-03T20:22:00+00:00,87.0,48.0,7.0,None,25.939667,84.0
6,DLVC1,2024-08-03T19:55:00+00:00,100.0,52.0,10.0,None,19.922581,97.0
7,ER,2024-08-03T20:00:00+00:00,73.0,62.0,9.0,None,67.953955,73.0
8,FGMC1,2024-08-03T19:35:00+00:00,90.0,49.0,4.0,None,24.941569,87.0
9,FHMC1,2024-07-30T15:05:00+00:00,66.0,57.0,1.0,None,74.364506,NaN
